This notebook analyzes the data from the power meter to determine the fraction of the time that power is available on five microgrids in the Lake Sentani area.

# Data cleaning

TODO: decide if the Data Cleaning code should be published.

The raw data files are in Microsoft Excel format and are created by the metering software.

Issues:
- the excel files contain analysis graphs in the header
- data is reported in reverse chronological order with latest data first in the file
- when communication is lost, there is a "communication lost" string inserted into the power reading
- there are "power down" and "power up" readings inserted into the kWh export power reading when the generator turns off
- I have observed non-monotonicity in data that should be monotonic

These files need to be turned into well-behaved CSV files.
Ideally, this would be completely scripted using libraries for reading Excel files and exporting CSV files.
However, the files as created by the metering software cannot be read by some python Excel libraries.
I assume this is because the Excel files deviate from the expected format.

Each of the months of data are read in as Excel files, cleaned of communication messages, and converted to a list of data frames.
These data frames are then concatenated, sorted by the date time index and saved to disk as a CSV file.

In [9]:
# TODO: check for non-numeric entries
# TODO: maybe report how many events removed?
# TODO: look out for rows with null data
# TODO: set permissions to read-only for the output CSV files

import pandas as pd
import WP19_analysis

prefix = 'Diesel_Data/raw_data/'

for rfd in WP19_analysis.raw_file_data:
    energy_dfs = []
    message_dfs = []
    files = rfd['files']
    skiprows = rfd['skiprows']
    output_file = rfd['output_file']
    print(output_file)
    for f,sr in zip(files, skiprows):
        # convert excel file to temporary csv file on disk
        print(prefix + f)
        excel_data = pd.read_excel(prefix + f, skiprows=sr)
        assert 'Timestamp' in excel_data.columns, 'check your skiprows, header not found'
        excel_data.to_csv('temp.csv')

        # remove messages from csv file string object
        temp_csv = open('temp.csv')
        file_string = ''
        messages = 'date,index,message\n'
        omit_strings = ['Power Up', 'Communication Lost', 'Power Down', ',,,,,,,,,,,']
        for line in temp_csv.readlines():
            if not any(omit in line for omit in omit_strings):
                file_string += line
            else:
                fields = line.split(',')
                #import pdb;pdb.set_trace()
                messages += ','.join((fields[1], fields[0], fields[3]))
                messages += '\n'


        # read in string object to dataframe, sort and add to dataframe list
        import io
        energy_data = pd.read_csv(io.StringIO(file_string),
                              index_col=1, 
                              parse_dates=[1], 
                              thousands=',')
        energy_data = energy_data.sort_index()
        energy_dfs.append(energy_data)
        
        message_data = pd.read_csv(io.StringIO(messages),
                              index_col=0, 
                              parse_dates=[0])
        message_data = message_data.sort_index()
        message_dfs.append(message_data)


    pd.concat(energy_dfs).to_csv(output_file)
    pd.concat(message_dfs).to_csv(rfd['village_name'] + '-messages.csv')

ajau.csv
Diesel_Data/raw_data/4. April/Data_Logs AJAU April 22 - 30.xls
WARNING *** file size (3625288) not 512 + multiple of sector size (512)
Diesel_Data/raw_data/5. May/Data_Logs AJAU MAY.xls
Diesel_Data/raw_data/6. June/Data_Logs AJAU June.xls
WARNING *** file size (13147260) not 512 + multiple of sector size (512)
Diesel_Data/raw_data/7. July/Data_Logs AJAU July.xls
WARNING *** file size (13790853) not 512 + multiple of sector size (512)
Diesel_Data/raw_data/8. Agustus/Data_Logs Ajau Agustus.xls
WARNING *** file size (12314552) not 512 + multiple of sector size (512)
asei.csv
Diesel_Data/raw_data/4. April/Data_Logs ASEI April 22 - 30.xls
WARNING *** file size (3531439) not 512 + multiple of sector size (512)
Diesel_Data/raw_data/5. May/Data_Logs ASEI MAY.xls
Diesel_Data/raw_data/6. June/Data_Logs ASEI June.xls
WARNING *** file size (12805177) not 512 + multiple of sector size (512)
Diesel_Data/raw_data/7. July/Data_Logs ASEI July.xls
atamali.csv
Diesel_Data/raw_data/4. April/Data_

Now that these have been processed, we can validate the files.

- TODO: what is the best way to import my validation code?  install a specific version or copy the code in repo?
- TODO: is the energy field monotonic?
- TODO: does the energy field have non-numeric entries?

These are the MD5 hashes created on 19 June 2017 for the energy data files.
Subsequent runs of the cleaning code above don't change the hashes.

    MD5 (ajau.csv) = bd95712a4ce565d18514f3360b9cd9d0
    MD5 (asei.csv) = 9c0241dc30b1016252efd7d2dfdc6acf
    MD5 (atamali.csv) = 8987744da6d1e3b85f26c304e7d3b9e5
    MD5 (ayapo.csv) = 145ebb04af288701b488563f9d69f8c0
    MD5 (kensio.csv) = bb2f8d983550c8354365381f32972ff9
    
These are the MD5 hashes created on 21 June 2017 for the message files.

    MD5 (ajau-messages.csv) = 641f05537199c2b0f018d9c8466ab0e1
    MD5 (asei-messages.csv) = 307e1324a6be53113ae75f3403653df5
    MD5 (atamali-messages.csv) = 2ebe8dd3556e9c1cba70c3a3e5c51023
    MD5 (ayapo-messages.csv) = 0fecace21d41da147f5e3fb973f8c5db
    MD5 (kensio-messages.csv) = a124f6a37949ecd1a50bb593d0880b03

In [10]:
# this uses the jupyter notebook syntax to make system calls
for rfd in WP19_analysis.raw_file_data:
    md5file = rfd['output_file']
    !md5 {md5file}

MD5 (ajau.csv) = bd95712a4ce565d18514f3360b9cd9d0
MD5 (asei.csv) = 9c0241dc30b1016252efd7d2dfdc6acf
MD5 (atamali.csv) = 8987744da6d1e3b85f26c304e7d3b9e5
MD5 (ayapo.csv) = 145ebb04af288701b488563f9d69f8c0
MD5 (kensio.csv) = bb2f8d983550c8354365381f32972ff9


In [11]:
for rfd in WP19_analysis.raw_file_data:
    md5file = rfd['village_name'] + '-messages.csv'
    !md5 {md5file}

MD5 (ajau-messages.csv) = 641f05537199c2b0f018d9c8466ab0e1
MD5 (asei-messages.csv) = 307e1324a6be53113ae75f3403653df5
MD5 (atamali-messages.csv) = 2ebe8dd3556e9c1cba70c3a3e5c51023
MD5 (ayapo-messages.csv) = 0fecace21d41da147f5e3fb973f8c5db
MD5 (kensio-messages.csv) = a124f6a37949ecd1a50bb593d0880b03
